<a href="https://colab.research.google.com/github/twloehfelm/SAR2020/blob/master/04%20-%20Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table width="100%">
    <tr>
        <td valign="top"><img src="https://cdn.ymaws.com/www.abdominalradiology.org/graphics/logo.jpg"/></td>
        <td valign="middle" align="right"><h1>SAR 2020<br/>AI Masters Class</h1></td>
    </tr>
    <tr>
      <td align="center" colspan=2><h1>Segmentation</h1></td>
    </tr>
</table>


We are going to build a liver segmentation tool using Facebook's detectron2 object identification algorithm trained on 18 CTs from the Combined Healthy Abdominal Organ Segmentation (CHAOS) Challenge. We'll test it on 2 CTs from the same challenge that are not included in the training set.
<br/><br/><br/>
**References to the original dataset and related publications:**

> <sub><sup>A.E. Kavur, M. A. Selver, O. Dicle, M. Barış,  N.S. Gezer. CHAOS - Combined (CT-MR) Healthy Abdominal Organ Segmentation Challenge Data (Version v1.03) [Data set]. Apr.  2019. Zenodo. http://doi.org/10.5281/zenodo.3362844 </sup></sub>

> <sub><sup>A.E. Kavur, N.S. Gezer, M. Barış, P.-H. Conze, V. Groza, D.D. Pham, et al. "CHAOS Challenge - Combined (CT-MR) Healthy Abdominal Organ Segmentation", arXiv pre-print, Jan. 2020. https://arxiv.org/abs/2001.06535</sup></sub>

> <sub><sup>A.E. Kavur, N.S. Gezer, M. Barış, Y.Şahin, S. Özkan, B. Baydar, et al.  "Comparison of semi-automatic and deep learning-based automatic methods for liver segmentation in living liver transplant donors", Diagnostic and  Interventional  Radiology,  vol. 26, pp. 11–21, Jan. 2020. https://doi.org/10.5152/dir.2019.19025</sup></sub>

# Install required packages

In [0]:
# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html > /dev/null
!pip install cython pyyaml==5.1 > /dev/null
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI' > /dev/null
import torch, torchvision
torch.__version__
!gcc --version

#install detectron2:
!pip3 install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

#install pydicom
!pip3 install -q pydicom

In [0]:
# Download liver seg training data to root directory
!wget -q https://www.dropbox.com/s/ajg922tunqptd2z/chaos_train.zip
!unzip chaos_train.zip > /dev/null

In [0]:
## You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer, default_argument_parser, default_setup, launch
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode

from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_train_loader, build_detection_test_loader
from detectron2.data import transforms as T
from detectron2.data import detection_utils as utils

# Imports for liver seg
import os
from pathlib import Path
import json
import pydicom
import pycocotools
import png
import matplotlib.pyplot as plt
import copy

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  # opencv is pre-installed on colab
  # colab cannot use the standard imshow due to some html/web limitation
  from google.colab.patches import cv2_imshow as imshow
else:
  #!pip install opencv-python
  from matplotlib.pyplot import imshow
  %matplotlib inline

In [0]:
!rm -rf ./__MACOSX
!rm -rf ./chaos_train.zip
!rm -rf ./sample_data
ROOT_PATH = Path('/content')
chaos = ROOT_PATH/'chaos_train'
pts_dir = chaos/'CT'

# Prepare the dataset

In [0]:
# Create lists of DICOMs and segmentation files for the training and validation datasets
dicoms_train = list()
segs_train = list()
dicoms_val = list()
segs_val = list()

# Randomly split patients 90/10 into training/validation. 20 studies, so 18 train and 2 val
pts = [x for x in pts_dir.iterdir() if x.is_dir()]
random.seed(716)
random.shuffle(pts)
train_pts = pts[:18]
val_pts = pts[18:]

# For each patient, add DICOMs and segmentation files to the respective lists
# N.B. DICOM and seg file names must sort in the same order
for p in train_pts:
  dicoms_train += sorted([x for x in (p/'DICOM_anon').iterdir() if x.is_file()])
  segs_train += sorted([x for x in (p/'Ground').iterdir() if x.is_file()])
for p in val_pts:
  dicoms_val += sorted([x for x in (p/'DICOM_anon').iterdir() if x.is_file()])
  segs_val += sorted([x for x in (p/'Ground').iterdir() if x.is_file()])

In [0]:
def mapper(dataset_dict):
  """
  Args:
      dataset_dict (dict): Metadata of one image, in detectron2 Dataset format.

  Returns:
      dict: a format that builtin models in detectron2 accept
  """
  # Implement a mapper, similar to the default DatasetMapper, but with your own customizations
  dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
  ds = pydicom.dcmread(dataset_dict["file_name"])
  image = ds.pixel_array
  # Convert pixel values to Hounsfield units
  image = image*ds.RescaleSlope + ds.RescaleIntercept
  image, transforms = T.apply_transform_gens([T.RandomBrightness(0.8, 1.2), T.RandomContrast(0.8, 1.2)], image)
  dataset_dict["image"] = torch.as_tensor(image.astype("float32"))

  annos = [
      utils.transform_instance_annotations(obj, transforms, image.shape[:2])
      for obj in dataset_dict.pop("annotations")
      if obj.get("iscrowd", 0) == 0
  ]
  instances = utils.annotations_to_instances(annos, image.shape[:2], mask_format='bitmask')
  dataset_dict["instances"] = utils.filter_empty_instances(instances)
  return dataset_dict

class LiverTrainer(DefaultTrainer):
    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        return build_detection_test_loader(cfg, dataset_name, mapper=mapper)

    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=mapper)


In [0]:
def load_mask(image_path):
    """Load masks for the given image.
    Mask is a binary true/false map of the same size as the image.
    args:
      image_path = complete path to segmentation file
    returns:
      boolean array of the liver segmentation mask 
    """
    mask = plt.imread(str(image_path))
    return mask.astype(np.bool)

def bbox(img):
    """Generates a bounding box from a segmentation mask
    Bounding box is given as coordinates of upper left and lower right corner
    args:
      img = boolean array of a segmentation mask
    returns:
      coordinates of upper left and lower right corner 
    """
    try:
      x,y = np.where(img)
    except ValueError:
      return None
    if x.size != 0:
      bbox = y.min(), x.min(), y.max(), x.max()
    else:
      bbox = None
    return bbox

# from fastai2 medical imaging
def windowed(px, w, l):
    """Windows a pixel_array of Houndfield units
    args:
      px = pixel array in Houndfield units
      w = window width (HU range)
      l = window level (center point)
    returns:
      pixel_array convered to the given window/level
    """
    if type(w) == pydicom.multival.MultiValue:
      w = w[0]
    if type(l) == pydicom.multival.MultiValue:
      l = l[0]
    px_min = l - w//2
    px_max = l + w//2
    px[px<px_min] = px_min
    px[px>px_max] = px_max
    return (px-px_min) / (px_max-px_min)

# Used in the DatasetCatalog.register call
def get_liver_dicts(train_or_val):
  """Builds a dataset_dict for detectron2
    args:
      train_or_val = string 'train' or 'val' indicating whether to return
        the training or validation dataset_dict
    returns:
      dataset_dict with each element of the training or validation dataset
    """
  if train_or_val == "train":
    dicoms = dicoms_train
    segs = segs_train
  elif train_or_val == "val":
    dicoms = dicoms_val
    segs = segs_val

  dataset_dicts = []
  for idx, d in enumerate(dicoms):
    record = {}
    
    filename = str(d)
    ds = pydicom.dcmread(filename)
    height, width = ds.Rows, ds.Columns

    # Mininum required fields for each element in the dict
    record["file_name"] = filename  # Full path to image file
    record["image_id"] = idx        # Index of file (unique serial number)
    record["height"] = height       # Image dimension
    record["width"] = width         # Image dimension

    try:
      mask = load_mask(str(segs[idx]))
    except IndexError:
      mask = None

    # Add list of segmentation object(s)
    objs = []
    if bbox(mask) is not None:
      obj = {
          "bbox": bbox(mask),
          "bbox_mode": BoxMode.XYXY_ABS,
          "segmentation": pycocotools.mask.encode(np.asarray(mask, order="F")), # Convert binary mask to RLE format
          "category_id": 0,
          "is_crowd": 0
      }
      objs.append(obj)
    record["annotations"] = objs
    dataset_dicts.append(record)
  return dataset_dicts


In [0]:
# Clear existing DatasetCatalog and then register the training and validation datasets
DatasetCatalog.clear()  
for d in ["train", "val"]:
    DatasetCatalog.register("liver_" + d, lambda d=d: get_liver_dicts(d))
    MetadataCatalog.get("liver_" + d).set(thing_classes=["liver"])
liver_metadata = MetadataCatalog.get("liver_train")

# Verify Data Loading
To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:



In [0]:
dataset_dicts = get_liver_dicts("train")

In [ ]:
# Show examples of training data: mask + DICOM
for d in random.sample(dataset_dicts, 3):
  try:
    # Decode the binary mask from the RLE string
    mask = pycocotools.mask.decode(d["annotations"][0]["segmentation"])
    # Extract pixel array from DICOM and convert to default W/L
    ds=pydicom.dcmread(d["file_name"])
    im=ds.pixel_array
    im=im*ds.RescaleSlope + ds.RescaleIntercept
    im = windowed(im, ds.WindowWidth, ds.WindowCenter)
    
    #Plot the mask and windowed DICOM
    fig, axs = plt.subplots(1, 2, figsize=(10, 5), sharey=True)
    axs[0].imshow(mask, cmap=plt.cm.binary_r)
    axs[1].imshow(im, cmap='gray')
    plt.show()
  except IndexError:
    continue

In [0]:
# Same as previous, but use detectron's default display format with overlaid mask
for d in random.sample(dataset_dicts, 3):
    ds=pydicom.dcmread(d["file_name"])
    im=ds.pixel_array
    im=im*ds.RescaleSlope + ds.RescaleIntercept
    im = windowed(im, ds.WindowWidth, ds.WindowCenter)
    im = np.stack((im,) * 3, -1)
    im=im*255
    v = Visualizer(im[:, :, ::-1],
                   metadata=liver_metadata, 
                   scale=0.8
    )
    v = v.draw_dataset_dict(d)
    imshow(v.get_image()[:, :, ::-1])

# Train

Now, let's fine-tune a coco-pretrained R50-FPN Mask R-CNN model on the liver dataset. It takes ~2 minutes to train 300 iterations on Colab.


In [0]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("liver_train",)
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo

# These three SOLVER parameters are probably the best places to start tweaking to modify performance
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.001  # Can experiment with differnt base learning rates
cfg.SOLVER.MAX_ITER = 500    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this simple dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (liver)

cfg.INPUT.FORMAT = "F" #32-bit single channel floating point pixels
cfg.INPUT.MASK_FORMAT = "bitmask" # Needed to change this from the default "polygons"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = LiverTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [0]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [0]:
%reload_ext tensorboard

# Inference & evaluation using the trained model
Now, let's run inference with the trained model on the validation dataset. First, let's create a predictor using the model we just trained:



In [0]:
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

class LiverPredictor:
    """
    Create a simple end-to-end predictor with the given config that runs on
    single device for a single input image.
    Compared to using the model directly, this class does the following additions:
    1. Load checkpoint from `cfg.MODEL.WEIGHTS`.
    2. Always take BGR image as the input and apply conversion defined by `cfg.INPUT.FORMAT`.
    3. Apply resizing defined by `cfg.INPUT.{MIN,MAX}_SIZE_TEST`.
    4. Take one input image and produce a single output, instead of a batch.
    If you'd like to do anything more fancy, please refer to its source code
    as examples to build and use the model manually.
    Attributes:
        metadata (Metadata): the metadata of the underlying dataset, obtained from
            cfg.DATASETS.TEST.
    Examples:
    .. code-block:: python
        pred = DefaultPredictor(cfg)
        inputs = cv2.imread("input.jpg")
        outputs = pred(inputs)
    """

    def __init__(self, cfg):
        self.cfg = cfg.clone()  # cfg can be modified by model
        self.model = build_model(self.cfg)
        self.model.eval()
        self.metadata = MetadataCatalog.get(cfg.DATASETS.TEST[0])

        checkpointer = DetectionCheckpointer(self.model)
        checkpointer.load(cfg.MODEL.WEIGHTS)

        self.transform_gen = T.ResizeShortestEdge(
            [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST
        )

        self.input_format = cfg.INPUT.FORMAT
        
    def __call__(self, original_image):
        """
        Args:
            original_image (np.ndarray): a single channel image.
        Returns:
            predictions (dict):
                the output of the model for one image only.
                See :doc:`/tutorials/models` for details about the format.
        """
        with torch.no_grad():  # https://github.com/sphinx-doc/sphinx/issues/4258
            height, width = original_image.shape[:2]
            image = original_image
            image = torch.as_tensor(image.astype("float32"))

            inputs = {"image": image, "height": height, "width": width}
            predictions = self.model([inputs])[0]
            return predictions

In [0]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("liver_val", )
predictor = LiverPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

In [0]:
dataset_dicts = get_liver_dicts("val")
for d in random.sample(dataset_dicts, 10): 
    ds=pydicom.dcmread(d["file_name"])
    im=ds.pixel_array
    im=im*ds.RescaleSlope + ds.RescaleIntercept
    outputs = predictor(im)
    im = windowed(im, ds.WindowWidth, ds.WindowCenter)
    im = np.stack((im,) * 3, -1)
    im=im*255
    v = Visualizer(im[:, :, ::-1],
                   metadata=liver_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    imshow(v.get_image()[:, :, ::-1])